# Supervisor Multi Agent - Long Term Memory

## tools

In [ ]:
from langchain_core.tools import tool
from langchain_community.document_loaders import WikipediaLoader
from langchain_tavily import TavilySearch
from typing import List, Dict
import requests
import yfinance as yf
from pprint import pformat
from datetime import datetime

FORBIDDEN_KEYWORDS = {
    "403 forbidden", "access denied", "captcha",
    "has been denied", "not authorized", "verify you are a human"
}

@tool
def web_search(query: str, max_results: int = 5) -> Dict[str, List[Dict[str, str]]]:
    """
    General-purpose web search.

    Use when you need recent or broader information from the web to answer the user's request
    (e.g., discover relevant entities, find supporting context, or gather up-to-date references).

    Parameters:
    - query (str): The search query in plain language.
    - max_results (int): Number of results to return (default 5, max 10).

    Returns:
    - {"results": [{"title": str, "url": str, "snippet": str}, ...]}

    Example:
    - query: "emerging AI hardware companies"
    """
    max_results = max(1, min(max_results, 10))
    tavily = TavilySearch(max_results=max_results)
    raw = tavily.invoke({"query": query})

    results = [
        {k: v for k, v in page.items() if k != "raw_content"}  # drop heavy field
        for page in raw["results"]
        if not any(
            k in ((page.get("content") or "").lower())
            for k in FORBIDDEN_KEYWORDS
        )
    ]

    return {"results": results}


@tool
def wiki_search(topic: str, max_results: int = 5) -> dict:
    """
    Fetch a concise encyclopedic summary for a single entity or topic.

    When to use:
      - You need neutral background about a company, product, person, or concept.

    How to format `topic` (VERY IMPORTANT):
      - Pass a short, Wikipedia-friendly title or entity name.
      - Avoid questions or long queries. Prefer canonical forms.
      - If you have noisy text, reduce it to the key noun phrase.

    Good examples:
      - "NVIDIA", "OpenAI", "Large language model", "Electric vehicle"
    Avoid:
      - "What is NVIDIA and why is it important?", "tell me about AI chips 2025"

    Parameters:
      - topic (str): Canonical page title or concise entity/topic.
    """
    max_results = max(1, min(max_results, 10))
    wiki = WikipediaLoader(query=topic, load_max_docs=max_results)
    raw = wiki.load()

    results = [
      {
        "title": doc.metadata["title"],
        "summary": doc.metadata["summary"],
        "source": doc.metadata["source"]
      }
      for doc in raw
    ]

    return {"results": results}


@tool("lookup_stock")
def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


@tool("fetch_stock_data")
def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}


@tool
def place_order(
    symbol: str,
    action: str,
    shares: int,
    limit_price: float,
    order_type: str = "limit",
) -> dict:
    """
    Execute a stock order.

    Parameters:
    - symbol: Ticker
    - action: "buy" or "sell"
    - shares: Number of shares to trade (pre-computed by the agent)
    - limit_price: Limit price per share
    - order_type: Order type, default "limit"

    Returns:
    - status: Execution result (simulated)
    - symbol
    - shares
    - limit_price
    - total_spent
    - type: Order type used
    - action
    """
    total_spent = round(int(shares) * limit_price, 2)
    return {
        "status": "filled",
        "symbol": symbol,
        "shares": int(shares),
        "limit_price": limit_price,
        "total_spent": total_spent,
        "type": order_type,
        "action": action,
    }


@tool
def current_timestamp() -> dict:
    """
    Return the current local timestamp.

    Returns:
    - {"iso": str, "epoch": int, "tz": str}
      where:
      - iso: ISO 8601 string with timezone offset
      - epoch: Unix epoch seconds
      - tz: timezone name/offset
    """
    now = datetime.now().astimezone()
    return {
        "iso": now.isoformat(),
        "epoch": int(now.timestamp()),
        "tz": str(now.tzinfo),
    }

## research_agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field
from IPython.display import Image, display

RESEARCH_SYSTEM_MESSAGE = """
You are a Research Agent that recommends ONE promising company for investment based on user requests.

Find a company that matches the user's theme/sector. Use tools to verify information. Be factual and concise.

Rules:
- Recommend exactly ONE company that is publicly tradable
- **Important!** ensure that the company you recommend is is publicly tradable!
- Make 2-3 tool calls maximum
- Don't place trades or fabricate data
- End with: CHOSEN_COMPANY: <Company Name>

Output a 1-2 sentence explanation followed by the company name.
"""

research = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[web_search, wiki_search],
    prompt=RESEARCH_SYSTEM_MESSAGE,

    name="research"
)

display(Image(research.get_graph().draw_mermaid_png()))

## Memory Tools & Store Initialization

In [ ]:
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig
from langgraph.config import get_store
from langgraph.store.memory import InMemoryStore

import uuid

_GLOBAL_STORE = None

def initialize_store(store):
    global _GLOBAL_STORE
    _GLOBAL_STORE = store

def get_global_store():
    return _GLOBAL_STORE

initialize_store(InMemoryStore())


@tool
def get_order_history(config: RunnableConfig) -> list:
    """
    Retrieves past investment orders for the current user.
    
    Returns:
        A list of past orders with details including order_id, timestamp, symbol, shares, and price
        
    Example Usage: 
        Review previous investments before recommending new ones
    """
    user_id = config["configurable"].get("user_id")
    namespace = ("ledger", user_id)
    store = get_global_store()
    items = store.search(namespace)
    return [item.value for item in items]
    

@tool
def add_order_to_history(symbol: str, shares: int, price: float, config: RunnableConfig) -> dict:
    """
    Records a new investment order in the user's order history.
    
    Args:
        symbol: Stock ticker symbol (e.g., 'AAPL', 'MSFT')
        shares: Number of shares purchased or sold (positive for buy, negative for sell)
        price: Price per share in USD
        
    Returns:
        Dictionary containing the newly created order details including order_id and timestamp
        
    Example:
        To record a purchase of 10 shares of Apple at $190.50:
        add_order_to_history(symbol='AAPL', shares=10, price=190.50)
    """
    user_id = config["configurable"].get("user_id")
    namespace = ("ledger", user_id)
    store = get_global_store()

    order_id = str(uuid.uuid4())
    order = {
        "order_id": order_id,
        "ts": datetime.now().isoformat(),
        "symbol": symbol,
        "shares": shares,
        "price": price
    }
    store.put(namespace, order_id, order)

    return order

## portfolio_agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display

PORTFOLIO_SYSTEM_MESSAGE = """
You are a financial advisor that executes trades. Use tools to get market data and place orders.

Rules:
- Only execute trades for the EXACT company recommended by the research agent
- If the recommended company is not available for trading, report back without substituting alternatives
- Include specific parameters in your actions (symbol, shares, price)
- Use factual data, never fabricate information
- Do not make assumptions about alternative investments if the requested one is unavailable
- Maintain complete records of all successful trades for future reference and analysis
- NEVER provide any details of user's portfoliosummaries, if there is a request for report, only provide datas that might help building it and pass!
"""

portfolio = create_react_agent(
    model="openai:gpt-4o-mini",
    tools=[lookup_stock_symbol, fetch_stock_data_raw, place_order, add_order_to_history],
    prompt=PORTFOLIO_SYSTEM_MESSAGE,
    store=get_global_store(),
    name="portfolio"
)

display(Image(portfolio.get_graph().draw_mermaid_png()))

## supervisor

In [ ]:
from IPython.display import Image, display
from langgraph_supervisor import create_supervisor
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver


SUPERVISOR_SYSTEM_MESSAGE = """
You are a Financial Advisor Supervisor that coordinates specialized agents to fulfill user investment requests.

Your primary goal is to understand user needs and delegate tasks to the right specialist:

1. For investment ideas or research → Use research agent
   - This agent provides recommendations with supporting rationale

2. For executing investment decisions → Use portfolio agent
   - This agent handles the technical aspects of executing investments
   - Requires specific investment targets and budget
   - ALWAYS consult portfolio agent for current market prices when valuing assets

Core Principles:
- Persist until user requests are fully addressed
- When facing obstacles, adapt by seeking alternative paths
- Maintain continuity of user intent throughout the process
- Never leave a request unresolved without explicit user decision
- Proactively coordinate between agents to deliver complete solutions

Temporal Context:
- Begin by establishing current timeframe
- Consider temporal relevance in all recommendations
- Integrate time awareness into your analysis

Keep interactions efficient by asking only for essential information.
"""

supervisor = create_supervisor(
    agents=[research, portfolio],
    tools=[current_timestamp, get_order_history],
    model=ChatOpenAI(model="gpt-4o-mini"),
    prompt=SUPERVISOR_SYSTEM_MESSAGE,
    version="v2",
    output_mode="full_history",
    store=get_global_store(),
).compile(checkpointer=InMemorySaver())

display(Image(supervisor.get_graph().draw_mermaid_png()))

## testing

In [ ]:
from langchain_core.messages import HumanMessage
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "user_id": "evgeny"
    }
}

response = supervisor.invoke({"messages": [
    HumanMessage(content="""Do a research and invest $1,000 in the electric vehicle industry""")
]}, config)

for message in response['messages']:
    message.pretty_print()

In [ ]:
from langchain_core.messages import HumanMessage
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "user_id": "evgeny"
    }
}

response = supervisor.invoke({"messages": [
    HumanMessage(content="""Do a research and invest $1,000 in the AI industry""")
]}, config)

for message in response['messages']:
    message.pretty_print()

In [ ]:
from langchain_core.messages import HumanMessage
from textwrap import dedent
import uuid

config = {
    "configurable": {
        "thread_id": str(uuid.uuid4()),
        "user_id": "evgeny"
    }
}

response = supervisor.invoke({"messages": [
    HumanMessage(content=dedent("""
    I'd like to know how many shares I currently own of each stock and my total investment amount. 
    """))
]}, config)

for message in response['messages']:
    message.pretty_print()

## ledger state

In [ ]:
get_global_store().search(("ledger", "evgeny"))

## Reference Links

**1. Adding Memory to LangGraph Agents**

https://langchain-ai.github.io/langgraph/how-tos/memory/add-memory/

→ Guide for integrating persistent memory into LangGraph agents, including memory types, configuration options, and state management patterns.

**2. LangGraph BaseStore API Reference**

https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore

→ Technical documentation for the BaseStore interface, the foundation for implementing custom memory stores in LangGraph with persistence capabilities.